In [40]:
from sklearnex import patch_sklearn
patch_sklearn()
import numpy as np
import pandas as pd
import sklearn as sk
import xgboost as xgb
from kernals_1080 import *
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputRegressor
import joblib

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


get data

In [29]:
df = pd.read_csv('./time_profile.csv')
kernal_partion_source = df[kernals].to_numpy()
batch_size_train = df[['batch_size']].to_numpy()
target_train = df[['2080', 'titan', 'titanxp']].to_numpy()

dimensionality reduction for kernal usage

In [26]:
pca = PCA(n_components=64)
kernal_partion_train = pca.fit_transform(kernal_partion_source)
joblib.dump(pca, 'pca.pkl')

['pca.pkl']

In [33]:
input_train = np.concatenate((kernal_partion_train, batch_size_train), axis=1)

In [35]:
xgtrain = xgb.DMatrix(input_train, target_train)

In [42]:
param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}

In [44]:
num_round = 100
multioutputregressor = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:linear')).fit(input_train, target_train)

[02:33:20] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[02:33:21] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[02:33:21] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [45]:
multioutputregressor.score(input_train, target_train)

0.9999964514072937

In [49]:
multioutputregressor.predict(input_train[5].reshape(1, -1))

array([[0.4453172 , 0.44768044, 0.84611183]], dtype=float32)

In [50]:
target_train[5]

array([0.44529611, 0.44769551, 0.84595204])

In [ ]:
joblib.